In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Q1.) Perceptron Classifier

In [ ]:
class Perceptron (object) :

    def __init__(self , eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter=n_iter
        self.random_state = random_state

    def train(self, X, y):
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi)) # 𝜂 𝑦 C − I𝑦 C
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self.errors_ 

    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

### Q2.) Adaline Classifier

In [ ]:
class Adaline():
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
        self.w_ = None
        self.cost_ = []
        self.rgen = np.random.RandomState(self.random_state)

    
    def train(self, X, y):
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []
        for _ in range(self.n_iter):
            net_input = self.net_input(X)
            output = net_input
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self.cost_
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

### Q3.) Stochastic Gradient Descent(SGD) Classifier

In [ ]:
class AdalineWithSGD(Adaline):

    def _shuffle(self, X, y):
        r = self.rgen.permutation(len(y))
        return X[r], y[r]

    def _update_weights(self, xi, target):
        output = self.net_input(xi)
        error = (target - output)
        self.w_[1:] += self.eta * xi.dot(error)
        self.w_[0] += self.eta * error
        cost = 0.5 * error**2
        return cost

    def train(self, X, y):
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []
    
        for _ in range(self.n_iter):
            X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self.cost_

### Q4, Q5 & Q7.) Program to test Different Classifiers and Multiclass Classifier Using One-vs-Rest

In [ ]:
class tester(object):

    classifiers = ['perceptron', 'adaline', 'sgd']

    def __init__(self, classifier_name, data_location) -> None:
        self.classifier_name = classifier_name
        self.data_location = data_location  
        self.classifier = self.validateAndInstantiate()     

    def validateAndInstantiate(self):
        if self.classifier_name not in self.classifiers: # Checking if the classifier name is valid
            Exception('Classifier name must be one of: ' + str(self.classifiers))
        if self.classifier_name == 'perceptron':
            classifier = Perceptron(eta=0.1, n_iter=1500)
        elif self.classifier_name == 'adaline':
            classifier = Adaline(eta=0.00001, n_iter=500, random_state=1)
        elif self.classifier_name == 'sgd':
            classifier = AdalineWithSGD(eta=0.001, n_iter=100, random_state=1)
        return classifier
    
    def loadData(self, separator=',', class_column_index=-1, positive_class_value=0):
        # Importing the dataset
        df = pd.read_csv(self.data_location, sep=separator, header=None)

        # shuffle the data
        df = df.sample(frac=1).reset_index(drop=True)

        # Separate the data into features and the class column
        y = df.iloc[:, class_column_index].values

        # drop the class column
        X = df.drop(df.columns[class_column_index], axis=1)

        
        # get unique class value from dataframe; since its already randomized, we can just take the first value
        positive_class = np.sort(np.unique(y))[positive_class_value]

        print(positive_class)
        # 1 positive and other negative class
        y = np.where(y == positive_class, -1, 1) # Convert the class labels to two integer
        X = X.values

        # Feature Scaling
        # Normalize the data to have mean=0 and standard deviation=1
        for i in range(X.shape[1]):
            X[:, i] = (X[:, i] - X[:, i].mean()) / X[:, i].std()
        
        # Splitting the dataset into the Training set and Test set 
        percent = int(0.7 * len(y))
        self.X_train = X[:percent, :]
        self.y_train = y[:percent]
        self.X_test = X[percent:, :]
        self.y_test = y[percent:]

    
    def results(self):
        self.classifier.train(self.X_train, self.y_train)
        y_pred = self.classifier.predict(self.X_test)
        # Performance metrics: Accuracy
        accuracy = np.sum(y_pred == self.y_test) / len(self.y_test)
        # Print Accuracy in percentage
        print('Accuracy: ' + str(100 * np.mean((y_pred == self.y_test).astype(float))) + '%')
        
        print('The classifier failed to predict the class of {} samples'.format(np.sum(y_pred != self.y_test)))
        # print(pd.crosstab(self.y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
        return accuracy

    def train(self):
        return self.classifier.train(self.X_train, self.y_train) 

    def to_string(self):
        return self.classifier_name + ' ' + 'learning rate: ' + str(self.classifier.eta)

models = tester.classifiers

# (data_location, class_column_index, no_of_classes)
datasets = [('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',-1,3), 
    ('http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',0, 3)]

# iterate across all models and datasets and add the training_cost to subplots 

fig, axs = plt.subplots(3, 2)

accuracies = []
for i, model  in enumerate(models):
    accuracy_list_model = []
    for j, (dataset, class_col_idx, no_of_classes) in enumerate(datasets):
        testerObj = tester(model, dataset)

        ### Q7.) Multiclass Classification Using One-vs-Rest with SGD
        
        # pick class with highest confidence
        per_class_accuracy = 0
        per_class_cost  = []
        for k in range(no_of_classes):
            testerObj.loadData(class_column_index=class_col_idx, positive_class_value=k)
            training_cost = testerObj.train()
            accuracy = testerObj.results()

            if(accuracy > per_class_accuracy):
                per_class_accuracy = accuracy
                per_class_cost = training_cost

        # collect accuracy to a list
        accuracy_list_model.append(per_class_accuracy)  

        # Add the training_cost to subplots
        plt.subplot(len(models), len(datasets), models.index(model) * len(datasets) + datasets.index((dataset,class_col_idx,no_of_classes)) + 1)
        axs[i, j].plot(range(1, len(training_cost) + 1), training_cost, marker='o')
        axs[i, j].set_title(testerObj.to_string())
    accuracies.append(accuracy_list_model)

plt.xlabel('Epochs')
plt.ylabel('Average Cost')
plt.show()
                


# Print the test accuracy for each model and dataset
table = plt.table(cellText=accuracies, rowLabels=models, colLabels=['iris','wine'], loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
plt.axis('off')
plt.title('Test Accuracy for each model and dataset')
plt.show()